# Prompt Caching: 프롬프트 캐싱

## 예제 1: 도구 및 다중 턴 대화 캐싱하기

In [1]:
import os
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from openai import OpenAI

MODEL = "gpt-4.1-mini"
client = OpenAI()

In [3]:
import time
import json

# 모델이 사용할 수 있는 함수(도구)들을 정의한다.
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "고객 주문의 배송 날짜를 가져온다. 고객이 '제 소포는 어디 있나요'라고 물을 때처럼 배송 날짜를 알아야 할 때마다 이 함수를 호출한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {"type": "string", "description": "고객의 주문 ID다."},
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cancel_order",
            "description": "아직 배송되지 않은 주문을 취소한다. 고객이 주문 취소를 요청할 때 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {"type": "string", "description": "고객의 주문 ID다."},
                    "reason": {"type": "string", "description": "주문 취소 사유다."}
                },
                "required": ["order_id", "reason"],
                "additionalProperties": False
            }
        }
    },
    # ... (다른 도구 정의들) ...
    {
        "type": "function",
        "function": {
            "name": "update_payment_method",
            "description": "아직 완료되지 않은 주문의 결제 수단을 업데이트한다. 고객이 결제 정보를 변경하고 싶을 때 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {"type": "string", "description": "고객의 주문 ID다."},
                    "payment_method": {
                        "type": "object",
                        "properties": {
                            "card_number": {"type": "string", "description": "새 신용카드 번호다."},
                            "expiry_date": {"type": "string", "description": "새 신용카드 유효 기간(MM/YY 형식)이다."},
                            "cvv": {"type": "string", "description": "새 신용카드 CVV 코드다."}
                        },
                        "required": ["card_number", "expiry_date", "cvv"],
                        "additionalProperties": False
                    }
                },
                "required": ["order_id", "payment_method"],
                "additionalProperties": False
            }
        }
    }
]

# 시스템 메시지를 통해 어시스턴트의 역할과 행동 지침(가드레일)을 상세히 정의한다.
# 이 정적인 부분은 캐싱의 대상이 된다.
messages = [
    {
        "role": "system",
        "content": (
            "당신은 전문적이고, 공감 능력이 뛰어나며, 효율적인 고객 지원 어시스턴트다..."
            # ... (상세한 시스템 메시지 내용) ...
        )
    },
    {
        "role": "user",
        "content": (
            "안녕하세요, 3일 전에 주문했는데 언제 배송되는지 업데이트를 받지 못했습니다. "
            "배송 날짜 확인 좀 도와주시겠어요? 제 주문 번호는 #9876543210입니다. 이 물건이 급하게 필요해서 좀 걱정되네요."
        )
    }
]

# 두 번째 사용자 문의를 정의한다.
user_query2 = {
    "role": "user",
    "content": (
        "제 주문이 아직 배송되지 않았으니 취소하고 싶습니다. "
        "주문 번호는 #9876543210이고, 더 빨리 받으려고 현지에서 구매하기로 해서 취소해야 합니다. "
        "도와주시겠어요? 감사합니다!"
    )
}

# 제공된 메시지 기록과 도구로 API를 호출하는 함수다.
def completion_run(messages, tools):
    completion = client.chat.completions.create(
        model=MODEL,
        tools=tools,
        messages=messages,
        tool_choice="required" # 모델이 반드시 도구를 사용하도록 강제한다.
    )
    # 응답 객체를 JSON 형식으로 변환하여 반환한다.
    usage_data = json.dumps(completion.to_dict(), indent=4)
    return usage_data

# 두 번의 실행을 처리하는 메인 함수다.
def main(messages, tools, user_query2):
    # 실행 1: 초기 문의
    print("실행 1:")
    run1 = completion_run(messages, tools)
    print(run1)

    # 7초간 지연시킨다. 실제로는 즉시 다음 요청을 보낼 수 있다.
    time.sleep(7)

    # 메시지 기록에 두 번째 사용자 문의를 추가한다.
    messages.append(user_query2)

    # 실행 2: 추가된 문의와 함께
    print("\n실행 2:")
    run2 = completion_run(messages, tools)
    print(run2)

# 메인 함수를 실행한다.
main(messages, tools, user_query2)

실행 1:
{
    "id": "chatcmpl-C6SoWpZE6xOcUyYa0ZdIcSzf1trLR",
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "annotations": [],
                "tool_calls": [
                    {
                        "id": "call_613gfuqwhQnEk9oysimJJzhg",
                        "function": {
                            "arguments": "{\"order_id\":\"9876543210\"}",
                            "name": "get_delivery_date"
                        },
                        "type": "function"
                    }
                ]
            }
        }
    ],
    "created": 1755656764,
    "model": "gpt-4o-mini-2024-07-18",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": "fp_560af6e559",
    "usage": {
        "completion_tokens": 19,
      

## 예제 2: 이미지 캐싱하기

In [ ]:
# 이미지 URL들을 변수로 정의한다.
sauce_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/12-04-20-saucen-by-RalfR-15.jpg/800px-12-04-20-saucen-by-RalfR-15.jpg"
veggie_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Veggies.jpg/800px-Veggies.jpg"
eggs_url= "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Egg_shelf.jpg/450px-Egg_shelf.jpg"
milk_url= "https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Lactaid_brand.jpg/800px-Lactaid_brand.jpg"

# 여러 이미지를 포함하여 API를 호출하는 함수다.
def multiimage_completion(url1, url2, user_query):
    completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
        "role": "user",
        "content": [ # content 배열에 이미지와 텍스트를 함께 전달한다.
            {
            "type": "image_url",
            "image_url": {
                "url": url1,
                "detail": "high" # 이미지 해상도 설정
            },
            },
            {
            "type": "image_url",
            "image_url": {
                "url": url2,
                "detail": "high"
            },
            },
            {"type": "text", "text": user_query}
        ],
        }
    ],
    max_tokens=300,
    )
    print(json.dumps(completion.to_dict(), indent=4))


def main(sauce_url, veggie_url, milk_url, eggs_url):
    # 실행 1: sauce_url과 veggie_url 사용
    multiimage_completion(sauce_url, veggie_url, "이 이미지들에 어떤 종류의 소스가 보이나요?")
    time.sleep(20) # 20초 지연

    # 실행 2: 동일한 URL 조합으로 다른 질문
    multiimage_completion(sauce_url, veggie_url, "이 이미지들에 어떤 종류의 채소가 보이나요?")
    time.sleep(20)

    # 실행 3: 다른 URL 조합으로 동일한 질문
    multiimage_completion(eggs_url, sauce_url, "이 이미지들에 어떤 종류의 소스가 보이나요?")


if __name__ == "__main__":
    main(sauce_url, veggie_url, milk_url, eggs_url)

{
    "id": "chatcmpl-C6Sq66QNYYzkfX2yomYHfUDFFh0oH",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "\uccab \ubc88\uc9f8 \uc774\ubbf8\uc9c0\uc5d0\uc11c \ubcfc \uc218 \uc788\ub294 \uc18c\uc2a4\ub294 \ud06c\uac8c \uc138 \uac00\uc9c0\uc785\ub2c8\ub2e4:\n1. \ud0a8\ucf54\ub9cc \uac04\uc7a5 (Kikkoman Soy Sauce)\n2. \uc6b0\uc2a4\ud130 \uc18c\uc2a4 (Worcester Sauce)\n3. \ud0c0\ubc14\uc2a4\ucf54 \uc18c\uc2a4 (Tabasco Pepper Sauce)\n\n\ub450 \ubc88\uc9f8 \uc774\ubbf8\uc9c0\uc5d0\ub294 \uc18c\uc2a4\uac00 \ubcf4\uc774\uc9c0 \uc54a\uace0 \ub2e4\uc591\ud55c \uc885\ub958\uc758 \uc2e0\uc120\ud55c \ucc44\uc18c\ub4e4\uc774 \uc9c4\uc5f4\ub418\uc5b4 \uc788\uc2b5\ub2c8\ub2e4.",
                "refusal": null,
                "role": "assistant",
                "annotations": []
            }
        }
    ],
    "created": 1755656862,
    "model": "gpt-4.1-mini-2025-04-14",
    "obj